# Setup

In [16]:
import pandas as pd
import re

# Import de dados

In [2]:
!ls

ml-25m	ml-25m.zip  Untitled.ipynb


In [3]:
!ls ml-25m

genome-scores.csv  links.csv   ratings.csv  tags.csv
genome-tags.csv    movies.csv  README.txt


In [124]:
df_movies = pd.read_csv('./ml-25m/movies.csv')

In [125]:
df_movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [126]:
!ls

f_movies.csv  ml-25m  ml-25m.zip  Untitled.ipynb


In [127]:
df_fmovies = pd.read_csv('f_movies.csv')

In [128]:
df_fmovies.head(1)

,Rank,Film,Year,Count,Running time\n(minutes)
0,1.0,Swearnet: The Movie,2014,935,112


# Join das tabelas

## Movies

In [129]:
df_movies.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [130]:
df_movies['title_processed'] = df_movies['title'].apply(lambda x: x.lower())
df_movies['title_processed'] = df_movies['title_processed'].apply(lambda x: x.replace(" ",""))
df_movies['title_processed'] = df_movies['title_processed'].apply(lambda x: re.sub('[^a-zA-Z]+', '',x))

In [131]:
df_movies.head(1)

,movieId,title,genres,title_processed
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toystory


## f_movies

In [132]:
df_fmovies.head(1)

,Rank,Film,Year,Count,Running time\n(minutes)
0,1.0,Swearnet: The Movie,2014,935,112


In [133]:
df_fmovies['title_processed'] = df_fmovies['Film'].apply(lambda x: x.lower())
df_fmovies['title_processed'] = df_fmovies['title_processed'].apply(lambda x: x.replace(" ",""))
df_fmovies['title_processed'] = df_fmovies['title_processed'].apply(lambda x: re.sub('[^a-zA-Z]+', '',x))

In [135]:
df_fmovies.shape

(165, 6)

## Join

In [139]:
df_final = df_fmovies.merge(df_movies, on = 'title_processed', how = 'inner')

In [140]:
df_final.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


In [141]:
df_final['movieId'].isna().sum()

0

In [142]:
df_final.shape

(173, 9)

In [143]:
df_final_bckp = df_final.copy()

## Consistência e limpeza de dados

In [144]:
df_final['title_processed'].value_counts()

stateproperty    4
badsanta         4
monster          4
skin             3
legend           3
                ..
gridlockd        1
stateofgrace     1
mynameisjoe      1
infamous         1
nextdayair       1
Name: title_processed, Length: 131, dtype: int64

In [145]:
df_final[df_final['title_processed'] == 'stateproperty']

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
8,13.0,State Property,2002,321,88,stateproperty,5138,State Property (2002),Action|Crime|Drama
9,13.0,State Property,2002,321,88,stateproperty,33132,State Property 2 (2005),Action|Crime|Drama
10,35.0,State Property 2,2005,271,94,stateproperty,5138,State Property (2002),Action|Crime|Drama
11,35.0,State Property 2,2005,271,94,stateproperty,33132,State Property 2 (2005),Action|Crime|Drama


In [146]:
df_final.drop(index = [9,10], inplace = True)

In [147]:
df_final[df_final['title_processed'] == 'stateproperty']

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
8,13.0,State Property,2002,321,88,stateproperty,5138,State Property (2002),Action|Crime|Drama
11,35.0,State Property 2,2005,271,94,stateproperty,33132,State Property 2 (2005),Action|Crime|Drama


In [149]:
df_final = df_final_bckp

In [150]:
df_final.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


### df_final parte 1: sem duplicadas

In [151]:
df_final['title_processed'].value_counts()

stateproperty    4
badsanta         4
monster          4
skin             3
legend           3
                ..
gridlockd        1
stateofgrace     1
mynameisjoe      1
infamous         1
nextdayair       1
Name: title_processed, Length: 131, dtype: int64

In [152]:
df_count = df_final['title_processed'].value_counts().to_frame('counts')

In [153]:
df_count[df_count['counts'] == 1]

,counts
dolemiteismyname,1
badboysii,1
boilerroom,1
allaboutthebenjamins,1
mids,1
...,...
gridlockd,1
stateofgrace,1
mynameisjoe,1
infamous,1


In [154]:
df_count[df_count['counts'] == 1].shape[0]

102

In [155]:
df_count[df_count['counts'] != 1].shape[0]

29

In [156]:
# Criação do df_final_1, com os filmes que não tem duplicadas
lista_1 = list(df_count[df_count['counts'] == 1].index)
df_final_1 = df_final[df_final['title_processed'].isin(lista_1)]

### df_final parte 2: criação e remoção de duplicadas

In [185]:
# Criação do df_final_2, com os filmes que não tem duplicadas
lista_2 = list(df_count[df_count['counts'] > 1].index)
df_parte_2 = df_final[df_final['title_processed'].isin(lista_2)]

In [186]:
df_parte_2.shape[0]

71

In [187]:
df_parte_2.head(10)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
8,13.0,State Property,2002,321,88,stateproperty,5138,State Property (2002),Action|Crime|Drama
9,13.0,State Property,2002,321,88,stateproperty,33132,State Property 2 (2005),Action|Crime|Drama
10,35.0,State Property 2,2005,271,94,stateproperty,5138,State Property (2002),Action|Crime|Drama
11,35.0,State Property 2,2005,271,94,stateproperty,33132,State Property 2 (2005),Action|Crime|Drama
13,16.0,Running Scared,2006,315,122,runningscared,2457,Running Scared (1986),Action|Comedy
14,16.0,Running Scared,2006,315,122,runningscared,5240,Running Scared (1980),Action|Drama
15,16.0,Running Scared,2006,315,122,runningscared,43921,Running Scared (2006),Action|Crime|Thriller
16,17.0,Sweet Sixteen,2002,313,106,sweetsixteen,6898,Sweet Sixteen (2002),Drama
17,17.0,Sweet Sixteen,2002,313,106,sweetsixteen,133990,Sweet Sixteen (1983),Horror|Thriller
28,31.0,Dirty,2005,280,97,dirty,105492,Dirty (2005),Crime|Drama|Thriller


In [188]:
# Coluna year_from_title: informação do ano retirada do título
df_parte_2['year_from_title'] = df_parte_2['title'].str.extract('\((\d+)\)$')

/home/heliomar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [189]:
df_parte_2.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres,year_from_title
8,13.0,State Property,2002,321,88,stateproperty,5138,State Property (2002),Action|Crime|Drama,2002


In [190]:
df_parte_2[df_parte_2['Year'] == df_parte_2['year_from_title']]

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres,year_from_title


In [191]:
df_parte_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 8 to 172
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rank                    71 non-null     float64
 1   Film                    71 non-null     object 
 2   Year                    71 non-null     int64  
 3   Count                   71 non-null     int64  
 4   Running time
(minutes)  71 non-null     int64  
 5   title_processed         71 non-null     object 
 6   movieId                 71 non-null     int64  
 7   title                   71 non-null     object 
 8   genres                  71 non-null     object 
 9   year_from_title         71 non-null     object 
dtypes: float64(1), int64(4), object(5)
memory usage: 6.1+ KB


In [192]:
df_parte_2['year_from_title'] = df_parte_2['year_from_title'].astype(str).astype(int)

/home/heliomar/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [193]:
df_parte_2[df_parte_2['Year'] == df_parte_2['year_from_title']].shape[0]

30

In [197]:
# Criação do df_final_2, que é o dataframe definitivo com os filmes duplicados
df_final_2 = df_parte_2[df_parte_2['Year'] == df_parte_2['year_from_title']].copy()

In [202]:
# Retirar os filmes que já tem a coluna Year == year_from_title
contador = 0
for film in list(df_parte_2[df_parte_2['Year'] == df_parte_2['year_from_title']]['Film']):
    if film in list(df_parte_2[df_parte_2['Year'] != df_parte_2['year_from_title']]['Film']):
        contador += 1
        df_parte_2.drop(df_parte_2[df_parte_2['Film'] == film].index, inplace = True)

/home/heliomar/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [203]:
contador

30

In [205]:
df_parte_2[df_parte_2['Year'] != df_parte_2['year_from_title']].shape[0]

2

In [206]:
df_parte_2[df_parte_2['Year'] != df_parte_2['year_from_title']]

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres,year_from_title
32,33.0,Cherry,2021,276,140,cherry,4092,Cherry 2000 (1987),Romance|Sci-Fi,1987
33,33.0,Cherry,2021,276,140,cherry,91048,Cherry (2010),Comedy|Drama,2010


## Criação do dataframe final

In [209]:
df_final_1.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


In [210]:
df_final_2.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres,year_from_title
8,13.0,State Property,2002,321,88,stateproperty,5138,State Property (2002),Action|Crime|Drama,2002


In [211]:
df_final_2.drop( columns = 'year_from_title', inplace = True)

In [212]:
df_final_1.columns == df_final_2.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [242]:
df = df_final_1.append(df_final_2, ignore_index = True)

In [243]:
df.shape

(132, 9)

In [216]:
df.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


In [217]:
df['Film'].value_counts()

Swearnet: The Movie           1
Snatch                        1
American Honey                1
Soul Men                      1
All About the Benjamins       1
                             ..
Zack and Miri Make a Porno    1
Looking for Eric              1
Suicide Kings                 1
Eddie Murphy Raw              1
The Night Before              1
Name: Film, Length: 132, dtype: int64

# Exploração inicial sobre df

In [218]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rank                    132 non-null    float64
 1   Film                    132 non-null    object 
 2   Year                    132 non-null    int64  
 3   Count                   132 non-null    int64  
 4   Running time
(minutes)  132 non-null    int64  
 5   title_processed         132 non-null    object 
 6   movieId                 132 non-null    int64  
 7   title                   132 non-null    object 
 8   genres                  132 non-null    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 9.4+ KB


In [219]:
df.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


In [222]:
df['Year'].value_counts(normalize= True)

2016    0.068182
2002    0.068182
1997    0.060606
2012    0.060606
1998    0.053030
2003    0.053030
2009    0.045455
1999    0.045455
2005    0.037879
2017    0.037879
2008    0.030303
1993    0.030303
2000    0.030303
2013    0.030303
2015    0.030303
2006    0.030303
1995    0.030303
2018    0.022727
2014    0.022727
2010    0.022727
2001    0.022727
2007    0.015152
2019    0.015152
2022    0.015152
1992    0.015152
1989    0.015152
1990    0.015152
2004    0.007576
2011    0.007576
1996    0.007576
1987    0.007576
2020    0.007576
1994    0.007576
1986    0.007576
1978    0.007576
1988    0.007576
1983    0.007576
Name: Year, dtype: float64

In [220]:
df['Count'].describe()

count    132.000000
mean     217.696970
std       90.274386
min      150.000000
25%      163.250000
50%      192.000000
75%      237.000000
max      935.000000
Name: Count, dtype: float64

In [223]:
df['genres'].value_counts(normalize= True)

Drama                                  0.159091
Comedy                                 0.121212
Crime|Drama                            0.113636
Action|Crime|Drama                     0.083333
Crime|Drama|Thriller                   0.068182
Comedy|Drama                           0.053030
Action|Comedy|Crime                    0.045455
Documentary                            0.022727
Comedy|Documentary                     0.022727
Drama|Romance                          0.022727
Comedy|Crime|Drama                     0.022727
Comedy|Crime                           0.015152
Crime|Thriller                         0.015152
Drama|Thriller                         0.015152
Crime                                  0.015152
Drama|War                              0.015152
Action|Thriller                        0.007576
Thriller                               0.007576
Comedy|Fantasy|Sci-Fi                  0.007576
Comedy|Crime|Thriller                  0.007576
Comedy|Musical                         0

In [224]:
df.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy


In [225]:
df['word_per_minute'] = df['Count']/df['Running time\n(minutes)']

In [226]:
df['word_per_minute'].describe()

count    132.000000
mean       1.986752
std        0.836193
min        0.877660
25%        1.480946
50%        1.818182
75%        2.333778
max        8.348214
Name: word_per_minute, dtype: float64

# df_rating

In [227]:
df_rating =pd.read_csv('./ml-25m/ratings.csv')

In [228]:
df_rating.shape

(25000095, 4)

In [229]:
df_rating.head(1)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [230]:
df_rating['movieId'].value_counts().head(10)

356     81491
318     81482
296     79672
593     74127
2571    72674
260     68717
480     64144
527     60411
110     59184
2959    58773
Name: movieId, dtype: int64

In [237]:
df_avg_rating = df_rating.groupby(by = 'movieId')['rating'].agg(Mean = 'mean', Count = 'count')
df_avg_rating.reset_index(inplace = True)

In [238]:
df_avg_rating.head(1)

,movieId,Mean,Count
0,1,3.893708,57309


In [239]:
df_avg_rating['movieId'].value_counts().head(5)

1         1
162388    1
162392    1
162394    1
162396    1
Name: movieId, dtype: int64

In [244]:
df_avg_rating.rename( columns = {'Count': 'Count_ratings'}, inplace = True)

In [245]:
df = df.merge(df_avg_rating, on = 'movieId', how = 'left')

In [246]:
df.head(1)

,Rank,Film,Year,Count,Running time\n(minutes),title_processed,movieId,title,genres,Mean,Count_ratings
0,1.0,Swearnet: The Movie,2014,935,112,swearnetthemovie,132132,Swearnet: The Movie (2014),Comedy,3.1,15


In [247]:
df['Count_ratings'].describe()

count      132.000000
mean      4201.492424
std       9902.520144
min          4.000000
25%        196.000000
50%        658.000000
75%       3334.500000
max      79672.000000
Name: Count_ratings, dtype: float64

In [248]:
df['Mean'].describe()

count    132.000000
mean       3.284216
std        0.520599
min        1.125000
25%        3.085717
50%        3.335093
75%        3.633288
max        4.188912
Name: Mean, dtype: float64